Sources:

[1] ["Who’s At The Keyboard? Authorship Attribution in Digital Evidence Investigations"](www.utica.edu/academic/institutes/ecii/publications/articles/B49F9C4A-0362-765C-6A235CB8ABDFACFF.pdf): good reference regarding number of authors and text volume.

[2] ["How a Computer Program Helped Show J.K. Rowling write A Cuckoo’s Calling"](https://www.scientificamerican.com/article/how-a-computer-program-helped-show-jk-rowling-write-a-cuckoos-calling/)


Notes:

- test approaches suggested in [1]
- measure average word lentgh per article, distribution of word lengths, the 100 most common words, distribution of character 4-grams, word bigrams
- test with support vector machines
- using word embeddings as a much rich representation


